In [ ]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import train_test_split

def prophet_forecasting(file_path, periods=200, freq='5T'):
    """
    Function to perform time series forecasting using Prophet.

    Parameters:
    - file_path: str, path to the CSV file containing the data
    - periods: int, number of periods to forecast
    - freq: str, frequency of the data (e.g., '5T' for 5 minutes)

    Returns:
    - combined_temp: DataFrame, original and forecasted values for temperature
    - combined_new_point: DataFrame, original and forecasted values for new_point
    - rmse_temp: float, root mean squared error for temperature predictions
    - rmse_new_point: float, root mean squared error for new_point predictions
    """
    # Load the data
    data = pd.read_csv(file_path)

    # Rename columns for convenience
    data.columns = ['ts', 'temp', 'new_point']

    # Convert 'ts' column to datetime and handle timezone offset
    data['ts'] = pd.to_datetime(data['ts'].str.replace(' Dubai', ''), errors='coerce')
    # Drop rows where datetime parsing failed
    data = data.dropna(subset=['ts'])

    # Clean temperature column and convert to numeric
    data['temp'] = data['temp'].str.replace('°C', '').astype(float)

    # Clean percentage column and convert to numeric
    data['new_point'] = data['new_point'].str.replace('%', '').astype(float)

    # Separate data for temperature and new_point
    df_temp = data[['ts', 'temp']].rename(columns={'ts': 'ds', 'temp': 'y'})
    df_new_point = data[['ts', 'new_point']].rename(columns={'ts': 'ds', 'new_point': 'y'})

    # Ensure 'ds' column is timezone-naive
    df_temp['ds'] = df_temp['ds'].dt.tz_localize(None)
    df_new_point['ds'] = df_new_point['ds'].dt.tz_localize(None)

    # Split the data into training and testing sets
    train_temp, test_temp = train_test_split(df_temp, test_size=0.2, shuffle=False)
    train_new_point, test_new_point = train_test_split(df_new_point, test_size=0.2, shuffle=False)

    # Initialize Prophet models with tuned hyperparameters
    model_temp = Prophet(seasonality_mode='additive', interval_width=0.95, changepoint_prior_scale=0.01)
    model_new_point = Prophet(seasonality_mode='additive', interval_width=0.95, changepoint_prior_scale=0.01)

    # Fit the models
    model_temp.fit(train_temp)
    model_new_point.fit(train_new_point)

    # Create future DataFrames for both temp and new_point
    future_temp = model_temp.make_future_dataframe(periods=periods, freq=freq)
    future_new_point = model_new_point.make_future_dataframe(periods=periods, freq=freq)

    # Predict the future values
    forecast_temp = model_temp.predict(future_temp)
    forecast_new_point = model_new_point.predict(future_new_point)

    # Compute RMSE for temp
    predicted_temp = forecast_temp['yhat'].values[-len(test_temp):]
    rmse_temp = np.sqrt(mean_squared_error(test_temp['y'], predicted_temp))

    # Compute RMSE for new_point
    predicted_new_point = forecast_new_point['yhat'].values[-len(test_new_point):]
    rmse_new_point = np.sqrt(mean_squared_error(test_new_point['y'], predicted_new_point))

    # Combine the original data with the forecasted values for temp and new_point
    combined_temp = pd.concat([df_temp, forecast_temp[['ds', 'yhat']].rename(columns={'yhat': 'y'})], axis=1)
    combined_new_point = pd.concat([df_new_point, forecast_new_point[['ds', 'yhat']].rename(columns={'yhat': 'y'})], axis=1)

    return combined_temp, combined_new_point, rmse_temp, rmse_new_point

# Example usage
file_path = r'C:\Users\syounas\OneDrive - Enova Facilities Management\Tasks\GitHub\HubgradeDataCleaning\Sana\Data\Train.csv'
combined_temp, combined_new_point, rmse_temp, rmse_new_point = prophet_forecasting(file_path)

print(f"RMSE for temp: {rmse_temp}")
print(f"RMSE for new_point: {rmse_new_point}")

# Save the combined DataFrames to separate CSV files
save_path_temp = r'C:\Users\syounas\OneDrive - Enova Facilities Management\Tasks\GitHub\HubgradeDataCleaning\Sana\Data\fp6.csv'
save_path_new_point = r'C:\Users\syounas\OneDrive - Enova Facilities Management\Tasks\GitHub\HubgradeDataCleaning\Sana\Data\fp7.csv'

combined_temp.to_csv(save_path_temp, index=False)
combined_new_point.to_csv(save_path_new_point, index=False)


In [1]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import train_test_split

def prophet_forecasting(df, length_of_missing_data, data_logging_interval, dqStart):
    """
    Function to perform time series forecasting using Prophet.

    Parameters:
    - df: DataFrame, input data with columns ['ts', 'temp', 'new_point']
    - length_of_missing_data: int, length of missing data
    - data_logging_interval: int, data logging interval
    - dqStart: not used in the function but kept for consistency with the original signature

    Returns:
    - df: DataFrame with predictions for all rows with missing columns. Index names as ts.
    """
    df = df.at[0, "his"]
    mt = df.set_index(["ts"])

    # Tag and filter rows with missing
    mt["status"] = mt.isna().any(axis=1)
    mt_predict = mt[mt["status"] == 1]
    X_predict = mt_predict.iloc[:, 0:1]

    # Filtered master table
    mt_train = mt.dropna()

    # Separate data for temperature and new_point
    df_temp = mt_train[['temp']].reset_index().rename(columns={'ts': 'ds', 'temp': 'y'})
    df_new_point = mt_train[['new_point']].reset_index().rename(columns={'ts': 'ds', 'new_point': 'y'})

    # Ensure 'ds' column is timezone-naive
    df_temp['ds'] = df_temp['ds'].dt.tz_localize(None)
    df_new_point['ds'] = df_new_point['ds'].dt.tz_localize(None)

    # Calculate periods based on length_of_missing_data and data_logging_interval
    periods = length_of_missing_data // data_logging_interval + 1

    # Split the data into training and testing sets
    train_temp, test_temp = train_test_split(df_temp, test_size=0.2, shuffle=False)
    train_new_point, test_new_point = train_test_split(df_new_point, test_size=0.2, shuffle=False)

    # Initialize Prophet models with tuned hyperparameters
    model_temp = Prophet(seasonality_mode='additive', interval_width=0.95, changepoint_prior_scale=0.01)
    model_new_point = Prophet(seasonality_mode='additive', interval_width=0.95, changepoint_prior_scale=0.01)

    # Fit the models
    model_temp.fit(train_temp)
    model_new_point.fit(train_new_point)

    # Create future DataFrames for both temp and new_point
    future_temp = model_temp.make_future_dataframe(periods=periods, freq=f'{data_logging_interval}T')
    future_new_point = model_new_point.make_future_dataframe(periods=periods, freq=f'{data_logging_interval}T')

    # Predict the future values
    forecast_temp = model_temp.predict(future_temp)
    forecast_new_point = model_new_point.predict(future_new_point)

    # Compute RMSE for temp
    predicted_temp = forecast_temp['yhat'].values[-len(test_temp):]
    rmse_temp = np.sqrt(mean_squared_error(test_temp['y'], predicted_temp))

    # Compute RMSE for new_point
    predicted_new_point = forecast_new_point['yhat'].values[-len(test_new_point):]
    rmse_new_point = np.sqrt(mean_squared_error(test_new_point['y'], predicted_new_point))

    # Making predictions on the same data or new data
    predict_temp = model_temp.predict(X_predict.rename(columns={X_predict.columns[0]: 'ds'}))
    predict_new_point = model_new_point.predict(X_predict.rename(columns={X_predict.columns[0]: 'ds'}))

    df_temp_pred = pd.DataFrame(data=predict_temp['yhat'].values, index=X_predict.index, columns=['temp_pred'])
    df_new_point_pred = pd.DataFrame(data=predict_new_point['yhat'].values, index=X_predict.index, columns=['new_point_pred'])

    df = pd.concat([df_temp_pred, df_new_point_pred], axis=1)
    
    return df

# # Example usage
# file_path = r'C:\Users\syounas\OneDrive - Enova Facilities Management\Tasks\GitHub\HubgradeDataCleaning\Sana\Data\Train.csv'
# data = pd.read_csv(file_path)
# forecast_df = prophet_forecasting(data, length_of_missing_data=200, data_logging_interval=5, dqStart=None)

# print(forecast_df.head())


In [16]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import train_test_split

def prophet_forecasting(df, length_of_missing_data, data_logging_interval, dqStart):
    """
    Function to perform time series forecasting using Prophet.

    Parameters:
    - df: DataFrame, input data with columns ['ts', 'temp', 'new_point']
    - length_of_missing_data: int, length of missing data
    - data_logging_interval: int, data logging interval
    - dqStart: not used in the function but kept for consistency with the original signature

    Returns:
    - df: DataFrame with predictions for all rows with missing columns. Index names as ts.
    """
    df = df.at[0, "his"]
    mt = df.set_index(["ts"])

    # Tag and filter rows with missing
    mt["status"] = mt.isna().any(axis=1)
    mt_predict = mt[mt["status"] == 1]
    X_predict = mt_predict.iloc[:, 0:1]

    # Filtered master table
    mt_train = mt.dropna()

    # Separate data for temperature and new_point
    df_temp = mt_train[['temp']].reset_index().rename(columns={'ts': 'ds', 'temp': 'y'})
    df_new_point = mt_train[['new_point']].reset_index().rename(columns={'ts': 'ds', 'new_point': 'y'})

    # Ensure 'ds' column is timezone-naive
    df_temp['ds'] = df_temp['ds'].dt.tz_localize(None)
    df_new_point['ds'] = df_new_point['ds'].dt.tz_localize(None)

    length_of_missing_data=400
    data_logging_interval=1

    # Calculate periods based on length_of_missing_data and data_logging_interval
    periods = (length_of_missing_data / data_logging_interval) + 1

    # Split the data into training and testing sets
    train_temp, test_temp = train_test_split(df_temp, test_size=0.2, shuffle=False)
    train_new_point, test_new_point = train_test_split(df_new_point, test_size=0.2, shuffle=False)

    # Initialize Prophet models with tuned hyperparameters
    model_temp = Prophet(seasonality_mode='additive', interval_width=0.95, changepoint_prior_scale=0.01)
    model_new_point = Prophet(seasonality_mode='additive', interval_width=0.95, changepoint_prior_scale=0.01)

    # Fit the models
    model_temp.fit(train_temp)
    model_new_point.fit(train_new_point)

    # Create future DataFrames for both temp and new_point
    future_temp = model_temp.make_future_dataframe(periods=periods, freq=f'{data_logging_interval}T')
    future_new_point = model_new_point.make_future_dataframe(periods=periods, freq=f'{data_logging_interval}T')

    # Predict the future values
    forecast_temp = model_temp.predict(future_temp)
    forecast_new_point = model_new_point.predict(future_new_point)

    # Compute RMSE for temp
    predicted_temp = forecast_temp['yhat'].values[-len(test_temp):]
    rmse_temp = np.sqrt(mean_squared_error(test_temp['y'], predicted_temp))

    # Compute RMSE for new_point
    predicted_new_point = forecast_new_point['yhat'].values[-len(test_new_point):]
    rmse_new_point = np.sqrt(mean_squared_error(test_new_point['y'], predicted_new_point))

    # Print RMSE values
    print(f"RMSE for temp: {rmse_temp}")
    print(f"RMSE for new_point: {rmse_new_point}")

    # Making predictions on the same data or new data
    predict_temp = model_temp.predict(X_predict.rename(columns={X_predict.columns[0]: 'ds'}))
    predict_new_point = model_new_point.predict(X_predict.rename(columns={X_predict.columns[0]: 'ds'}))

    df_temp_pred = pd.DataFrame(data=predict_temp['yhat'].values, index=X_predict.index, columns=['temp_pred'])
    df_new_point_pred = pd.DataFrame(data=predict_new_point['yhat'].values, index=X_predict.index, columns=['new_point_pred'])

    df = pd.concat([df_temp_pred, df_new_point_pred], axis=1)

    # Print the head of the resulting DataFrame
    print(df.head())
    
    return df


In [18]:
prophet_forecasting

<function __main__.prophet_forecasting(df, length_of_missing_data, data_logging_interval, dqStart)>

In [20]:
df= pd.read_csv(r'C:\Users\syounas\OneDrive - Enova Facilities Management\Tasks\GitHub\HubgradeDataCleaning\Sana\Data\Train.csv')

In [22]:
length_of_missing_data=400
data_logging_interval=1

In [24]:
prophet_forecasting

<function __main__.prophet_forecasting(df, length_of_missing_data, data_logging_interval, dqStart)>

In [26]:
import pandas as pd
import numpy as np

# Sample data creation
data = {
    'ts': pd.date_range(start='2023-01-01', periods=100, freq='D'),
    'temp': np.random.randn(100),
    'new_point': np.random.randn(100)
}
df = pd.DataFrame(data)

# Introducing some NaNs to simulate missing data
df.loc[10:20, 'temp'] = np.nan
df.loc[30:40, 'new_point'] = np.nan

# Converting to the expected input format
df['his'] = df.apply(lambda x: pd.DataFrame([{'ts': x['ts'], 'temp': x['temp'], 'new_point': x['new_point']}]), axis=1)

# Define the values for length_of_missing_data and data_logging_interval
length_of_missing_data = 400  # example value
data_logging_interval = 1  # example value

# Call the forecasting function with these parameters
forecast_df = prophet_forecasting(pd.DataFrame([{'his': df}]), length_of_missing_data, data_logging_interval, dqStart=None)

# Check the results
print(forecast_df.head())


09:14:59 - cmdstanpy - INFO - Chain [1] start processing
09:14:59 - cmdstanpy - INFO - Chain [1] done processing
09:14:59 - cmdstanpy - INFO - Chain [1] start processing
09:14:59 - cmdstanpy - INFO - Chain [1] done processing


TypeError: slice indices must be integers or None or have an __index__ method

In [28]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import train_test_split

def prophet_forecasting(df, length_of_missing_data, data_logging_interval, dqStart):
    """
    Function to perform time series forecasting using Prophet.

    Parameters:
    - df: DataFrame, input data with columns ['ts', 'temp', 'new_point']
    - length_of_missing_data: int, length of missing data
    - data_logging_interval: int, data logging interval
    - dqStart: not used in the function but kept for consistency with the original signature

    Returns:
    - df: DataFrame with predictions for all rows with missing columns. Index names as ts.
    """
    df = df.at[0, "his"]
    mt = df.set_index(["ts"])

    # Tag and filter rows with missing
    mt["status"] = mt.isna().any(axis=1)
    mt_predict = mt[mt["status"] == 1]
    X_predict = mt_predict.iloc[:, 0:1]

    # Filtered master table
    mt_train = mt.dropna()

    # Separate data for temperature and new_point
    df_temp = mt_train[['temp']].reset_index().rename(columns={'ts': 'ds', 'temp': 'y'})
    df_new_point = mt_train[['new_point']].reset_index().rename(columns={'ts': 'ds', 'new_point': 'y'})

    # Ensure 'ds' column is timezone-naive
    df_temp['ds'] = df_temp['ds'].dt.tz_localize(None)
    df_new_point['ds'] = df_new_point['ds'].dt.tz_localize(None)

    length_of_missing_data = 400
    data_logging_interval = 1

    # Calculate periods based on length_of_missing_data and data_logging_interval
    periods = int(length_of_missing_data / data_logging_interval) + 1

    # Split the data into training and testing sets
    train_temp, test_temp = train_test_split(df_temp, test_size=0.2, shuffle=False)
    train_new_point, test_new_point = train_test_split(df_new_point, test_size=0.2, shuffle=False)

    # Initialize Prophet models with tuned hyperparameters
    model_temp = Prophet(seasonality_mode='additive', interval_width=0.95, changepoint_prior_scale=0.01)
    model_new_point = Prophet(seasonality_mode='additive', interval_width=0.95, changepoint_prior_scale=0.01)

    # Fit the models
    model_temp.fit(train_temp)
    model_new_point.fit(train_new_point)

    # Create future DataFrames for both temp and new_point
    future_temp = model_temp.make_future_dataframe(periods=periods, freq=f'{data_logging_interval}T')
    future_new_point = model_new_point.make_future_dataframe(periods=periods, freq=f'{data_logging_interval}T')

    # Predict the future values
    forecast_temp = model_temp.predict(future_temp)
    forecast_new_point = model_new_point.predict(future_new_point)

    # Compute RMSE for temp
    predicted_temp = forecast_temp['yhat'].values[-len(test_temp):]
    rmse_temp = np.sqrt(mean_squared_error(test_temp['y'], predicted_temp))

    # Compute RMSE for new_point
    predicted_new_point = forecast_new_point['yhat'].values[-len(test_new_point):]
    rmse_new_point = np.sqrt(mean_squared_error(test_new_point['y'], predicted_new_point))

    # Print RMSE values
    print(f"RMSE for temp: {rmse_temp}")
    print(f"RMSE for new_point: {rmse_new_point}")

    # Making predictions on the same data or new data
    predict_temp = model_temp.predict(X_predict.rename(columns={X_predict.columns[0]: 'ds'}))
    predict_new_point = model_new_point.predict(X_predict.rename(columns={X_predict.columns[0]: 'ds'}))

    df_temp_pred = pd.DataFrame(data=predict_temp['yhat'].values, index=X_predict.index, columns=['temp_pred'])
    df_new_point_pred = pd.DataFrame(data=predict_new_point['yhat'].values, index=X_predict.index, columns=['new_point_pred'])

    df = pd.concat([df_temp_pred, df_new_point_pred], axis=1)

    # Print the head of the resulting DataFrame
    print(df.head())

    return df

# Sample data creation
data = {
    'ts': pd.date_range(start='2023-01-01', periods=100, freq='D'),
    'temp': np.random.randn(100),
    'new_point': np.random.randn(100)
}
df = pd.DataFrame(data)

# Introducing some NaNs to simulate missing data
df.loc[10:20, 'temp'] = np.nan
df.loc[30:40, 'new_point'] = np.nan

# Converting to the expected input format
df['his'] = df.apply(lambda x: pd.DataFrame([{'ts': x['ts'], 'temp': x['temp'], 'new_point': x['new_point']}]), axis=1)

# Define the values for length_of_missing_data and data_logging_interval
length_of_missing_data = 400  # example value
data_logging_interval = 1  # example value

# Call the forecasting function with these parameters
forecast_df = prophet_forecasting(pd.DataFrame([{'his': df}]), length_of_missing_data, data_logging_interval, dqStart=None)

# Check the results
print(forecast_df.head())


09:16:42 - cmdstanpy - INFO - Chain [1] start processing
09:16:42 - cmdstanpy - INFO - Chain [1] done processing
09:16:42 - cmdstanpy - INFO - Chain [1] start processing
09:16:42 - cmdstanpy - INFO - Chain [1] done processing


RMSE for temp: 1.031302303467329
RMSE for new_point: 1.2904003849087367


ValueError: Found NaN in column ds.

In [44]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import train_test_split

def prophet_forecasting(df, length_of_missing_data, data_logging_interval, dqStart):
    """
    Function to perform time series forecasting using Prophet.

    Parameters:
    - df: DataFrame, input data with columns ['ts', 'temp', 'new_point']
    - length_of_missing_data: int, length of missing data
    - data_logging_interval: int, data logging interval
    - dqStart: not used in the function but kept for consistency with the original signature

    Returns:
    - df: DataFrame with predictions for all rows with missing columns. Index names as ts.
    """
    df = df.at[0, "his"]
    mt = df.set_index(["ts"])

    # Tag and filter rows with missing
    mt["status"] = mt.isna().any(axis=1)
    mt_predict = mt[mt["status"] == 1]
    X_predict = mt_predict.iloc[:, 0:1]

    # Filtered master table
    mt_train = mt.dropna()

    # Separate data for temperature and new_point
    df_temp = mt_train[['temp']].reset_index().rename(columns={'ts': 'ds', 'temp': 'y'})
    df_new_point = mt_train[['new_point']].reset_index().rename(columns={'ts': 'ds', 'new_point': 'y'})

    # Ensure 'ds' column is timezone-naive
    df_temp['ds'] = df_temp['ds'].dt.tz_localize(None)
    df_new_point['ds'] = df_new_point['ds'].dt.tz_localize(None)

        # Calculate periods based on length_of_missing_data and data_logging_interval
    periods = int(length_of_missing_data / data_logging_interval) + 1

    # Split the data into training and testing sets
    train_temp, test_temp = train_test_split(df_temp, test_size=0.2, shuffle=False)
    train_new_point, test_new_point = train_test_split(df_new_point, test_size=0.2, shuffle=False)

    # Initialize Prophet models with tuned hyperparameters
    model_temp = Prophet(seasonality_mode='additive', interval_width=0.95, changepoint_prior_scale=0.00001)
    model_new_point = Prophet(seasonality_mode='additive', interval_width=0.95, changepoint_prior_scale=0.001)

    # Fit the models
    model_temp.fit(train_temp)
    model_new_point.fit(train_new_point)

    # Create future DataFrames for both temp and new_point
    future_temp = model_temp.make_future_dataframe(periods=periods, freq=f'{data_logging_interval}T')
    future_new_point = model_new_point.make_future_dataframe(periods=periods, freq=f'{data_logging_interval}T')

    # Predict the future values
    forecast_temp = model_temp.predict(future_temp)
    forecast_new_point = model_new_point.predict(future_new_point)

    # Compute RMSE for temp
    predicted_temp = forecast_temp['yhat'].values[-len(test_temp):]
    rmse_temp = np.sqrt(mean_squared_error(test_temp['y'], predicted_temp))

    # Compute RMSE for new_point
    predicted_new_point = forecast_new_point['yhat'].values[-len(test_new_point):]
    rmse_new_point = np.sqrt(mean_squared_error(test_new_point['y'], predicted_new_point))

    # Print RMSE values
    print(f"RMSE for temp: {rmse_temp}")
    print(f"RMSE for new_point: {rmse_new_point}")

    # Making predictions on the same data or new data
    X_predict = X_predict.dropna()  # Remove rows with NaN values in 'ds'
    predict_temp = model_temp.predict(X_predict.rename(columns={X_predict.columns[0]: 'ds'}))
    predict_new_point = model_new_point.predict(X_predict.rename(columns={X_predict.columns[0]: 'ds'}))

    df_temp_pred = pd.DataFrame(data=predict_temp['yhat'].values, index=X_predict.index, columns=['temp_pred'])
    df_new_point_pred = pd.DataFrame(data=predict_new_point['yhat'].values, index=X_predict.index, columns=['new_point_pred'])

    df = pd.concat([df_temp_pred, df_new_point_pred], axis=1)

    # Print the head of the resulting DataFrame
    print(df.head())

    return df

# Sample data creation
data = {
    'ts': pd.date_range(start='2023-01-01', periods=100, freq='D'),
    'temp': np.random.randn(100),
    'new_point': np.random.randn(100)
}
df = pd.DataFrame(data)

# Introducing some NaNs to simulate missing data
df.loc[10:20, 'temp'] = np.nan
df.loc[30:40, 'new_point'] = np.nan

# Converting to the expected input format
df['his'] = df.apply(lambda x: pd.DataFrame([{'ts': x['ts'], 'temp': x['temp'], 'new_point': x['new_point']}]), axis=1)

# Define the values for length_of_missing_data and data_logging_interval
length_of_missing_data = 400  # example value
data_logging_interval = 1  # example value

# Call the forecasting function with these parameters
forecast_df = prophet_forecasting(pd.DataFrame([{'his': df}]), length_of_missing_data, data_logging_interval, dqStart=None)

# Check the results
print(forecast_df.head())


09:21:26 - cmdstanpy - INFO - Chain [1] start processing
09:21:33 - cmdstanpy - INFO - Chain [1] done processing
09:21:33 - cmdstanpy - INFO - Chain [1] start processing
09:21:33 - cmdstanpy - INFO - Chain [1] done processing


RMSE for temp: 0.799515709598874
RMSE for new_point: 1.1167313712157632
            temp_pred  new_point_pred
ts                                   
2023-01-31 -59.896551       24.995766
2023-02-01 -59.896547       24.995764
2023-02-02 -59.896547       24.995764
2023-02-03 -59.896547       24.995764
2023-02-04 -59.896547       24.995764
            temp_pred  new_point_pred
ts                                   
2023-01-31 -59.896551       24.995766
2023-02-01 -59.896547       24.995764
2023-02-02 -59.896547       24.995764
2023-02-03 -59.896547       24.995764
2023-02-04 -59.896547       24.995764
